In [ ]:
from IPython.display import display
import pandas as pd
import altair as alt
from altair.utils.data import to_values

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [ ]:
conges = pd.read_csv("../../data/toflit18_all_flows.csv", sep=",")

In [ ]:
ports = ['Marseille',
'Le Havre',
'Dunkerque',
'Rouen',
'Bordeaux',
'Boulogne-sur-Mer',
'Marennes',
'Cap-Français',
'Nantes',
'La Rochelle',
'Saint-Malo',
'Toulon',
'Dieppe',
'Lorient',
'Honfleur',
'Marans',
'Sète',
'Bayonne',
'Tonnay-Charente',
'Arles']

In [ ]:
import csv
with open('../../data/toflit18_all_flows.csv') as f:
    reader = csv.DictReader(f)
#     customs_region_pos = reader.headers['customs_region']
#     value_pos = reader.headers['value']
#     partner_pos = reader.headers['partner_grouping']
    list_of_regions_values = []
    for row in reader:
        if row['customs_region'] in ports \
        and row['partner_grouping'] != 'France' \
        and row['year'] == '1789'\
        and row['export_import'] == 'Exports':
            list_of_regions_values.append(row)

In [ ]:
values_per_customs_region = {k:0 for k in ports}

for i in list_of_regions_values:
    if i['value']:
        values_per_customs_region[i['customs_region']] += float(i['value'])

In [ ]:
values_per_customs_region

In [ ]:
data = [{'customs_region':k, 'value':v} for k,v in values_per_customs_region.items()]

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Valeur des exports depuis les plus importantes directions de ferme",
    "mark": "bar",
    "data": {
        "values": data
    },
    "encoding": {
        "y": {
            "field": "value",
            "type": "quantitative", 
            "title": "value"
        },
        "x": {
            "field": "customs_region",
            "type": "nominal",
            "title": "customs region",
            "sort": "-y"
        }
    }
})